# Process and Plot the Duration of Historical Heat Waves and Cold Snaps by Balancing Authority

This notebook process the evolution of the hottest heat wave and the coldest cold snap for each historical year (1980-2019) in the TGW data. The analysis is currently done at the BA level but it could concievably be done by county or state.

In [1]:
# Start by importing the packages we need:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from glob import glob
from datetime import timedelta


## Set the Directory Structure

In [2]:
# Identify the data input and output directories:
data_input_dir =  '/Users/burl878/Documents/IMMM/Data/TELL/Production_Runs/tell_data/wrf_to_tell_data/historic/'
data_output_dir =  '/Users/burl878/Documents/IMMM/Data/TELL/Analysis/Heat_Wave_Cold_Snap_Duration/'
image_output_dir =  '/Users/burl878/Documents/IMMM/Images/TELL/Analysis/Heat_Wave_Cold_Snap_Duration/'


## Process the Historical Daily Minimum and Maximum Temperature Time Series

In [3]:
# Define a function to process the daily maximum and minimum temperature time series:
def process_daily_min_max_time_series(data_input_dir: str, data_output_dir: str, ba_to_process: str):
    
    # Loop over the years from the 1980 to 2019:
    for year in range(1980,2020,1):
        # Create the filename for the given BA and year combination:
        filename = (data_input_dir + ba_to_process + '_WRF_Hourly_Mean_Meteorology_' + str(year) + '.csv')
    
        # Read in the .csv file:
        met_df = pd.read_csv(filename)
    
        # Set the time variable as an index:
        met_df.index = pd.to_datetime(met_df['Time_UTC'])
        
        # Convert the temperature from Kelvin to Fahrenheit:
        met_df['T2'] = (1.8 * (met_df['T2'] - 273)) + 32
    
        # Compute the daily minimum and maximum temperature using resampling:
        temp_df = met_df.resample('D')['T2'].agg(['min', 'max']).reset_index()
        
        # Rename the variables for consistency:
        temp_df.rename(columns={'Time_UTC': 'Day', 'min': 'T2_Min', 'max': 'T2_Max'}, inplace=True)
        
        # Add a column with the year values to be used in grouping:
        temp_df['Year'] = temp_df['Day'].dt.strftime('%Y')
        
        # Reorder some of the variables:
        temp_df = temp_df[['Year', 'Day', 'T2_Min', 'T2_Max']].copy()
        
        # Set the day value as a datetime variable:
        temp_df['Day'] = pd.to_datetime(temp_df['Day'])
        
        # Aggregate the output into a new dataframe:
        if year == 1980:
           full_df = temp_df
        else:
           full_df = pd.concat([full_df, temp_df])
    
    #Initiate a counter to store the results:
    counter = 0;
    output_df = pd.DataFrame()
    
    # Loop over the years from 1981 to 2018:
    for year in range(1981,2019,1):
        # Iterate the counter by one:
        counter = counter + 1
            
        # Subset the full dataframe to just the year being processed:
        year_df = full_df.loc[full_df['Year'] == str(year)].copy()
        
        # Find coldest day of the year:
        coldest_day = year_df['Day'].loc[year_df['T2_Min'].idxmin()]
        
        # Add the coldest data and coldest temperature to the output file:
        output_df.loc[counter, 'Year'] = str(year)
        output_df.loc[counter, 'Coldest_Day'] = (str(coldest_day)).split(' ')[0]
        output_df.loc[counter, 'Coldest_Temp_F'] = year_df['T2_Min'].min().round(2)
        
        # Subset the full dataframe to a 20-day window around the coldest date:
        coldest_subset_df = pd.DataFrame(full_df['T2_Min'].loc[(full_df['Day'] >= (pd.to_datetime(coldest_day) - timedelta(days = 14))) 
                                                             & (full_df['Day'] <= (pd.to_datetime(coldest_day) + timedelta(days = 14)))].reset_index(drop=True))
        
        # Rename the column to reflect the year:
        coldest_subset_df.rename(columns={'T2_Min': str(year)}, inplace=True)
        
        # Aggregate the output into a new dataframe:
        if year == 1981:
           coldest_output_df = coldest_subset_df
        else:
           coldest_output_df = pd.concat([coldest_output_df, coldest_subset_df], axis=1)
    
        # Find hottest day of the year:
        hottest_day = year_df['Day'].loc[year_df['T2_Max'].idxmax()]
                      
        # Add the hottest day and hottest temperature to the output file:
        output_df.loc[counter, 'Hottest_Day'] = (str(hottest_day)).split(' ')[0]
        output_df.loc[counter, 'Hottest_Temp_F'] = year_df['T2_Max'].max().round(2)
        
        # Subset the full dataframe to a 20-day window around the hottest date:
        hottest_subset_df = pd.DataFrame(full_df['T2_Max'].loc[(full_df['Day'] >= (pd.to_datetime(hottest_day) - timedelta(days = 14))) 
                                                             & (full_df['Day'] <= (pd.to_datetime(hottest_day) + timedelta(days = 14)))].reset_index(drop=True))
        
        # Rename the column to reflect the year:
        hottest_subset_df.rename(columns={'T2_Max': str(year)}, inplace=True)
        
        # Aggregate the output into a new dataframe:
        if year == 1981:
           hottest_output_df = hottest_subset_df
        else:
           hottest_output_df = pd.concat([hottest_output_df, hottest_subset_df], axis=1)
    
    # Write out the dataframe to a .csv file:
    output_df.to_csv((os.path.join(data_output_dir + ba_to_process + '_Extreme_Days_1981_to_2018.csv')), sep=',', index=False)
    
    return coldest_output_df, hottest_output_df


In [5]:
coldest_output_df, hottest_output_df = process_daily_min_max_time_series(data_input_dir = data_input_dir, 
                                                                         data_output_dir = data_output_dir, 
                                                                         ba_to_process = 'CISO')

coldest_output_df


1981    1982    1983    1984    1985    1986    1987    1988    1989  \
0   46.238  35.564  47.624  43.952  45.464  49.262  43.934  49.586  41.936   
1   47.696  41.666  46.364  39.560  45.050  50.306  47.696  49.694  37.184   
2   48.686  45.212  49.118  43.466  43.826  51.386  45.626  46.130  37.382   
3   47.984  43.016  47.570  45.140  40.370  50.558  40.694  40.766  39.002   
4   51.890  40.622  49.172  47.246  39.002  52.970  38.030  41.774  39.866   
5   52.412  39.920  50.234  46.814  38.120  49.694  38.336  42.332  37.580   
6   48.236  42.476  49.928  45.176  38.174  45.878  35.006  39.902  40.388   
7   40.676  42.278  48.884  48.488  39.182  46.976  37.670  39.002  45.824   
8   38.174  44.690  49.784  46.634  41.864  44.402  39.056  40.622  45.644   
9   40.586  44.384  50.108  44.006  38.084  40.658  41.936  40.550  41.432   
10  44.060  42.296  50.720  36.572  38.930  42.584  42.134  37.454  38.588   
11  46.094  38.714  44.384  36.302  34.952  38.840  42.008  35.942  37.292   
12  43.592  40.208  45.752  38.444  32.990  35.600  39.488  38.966  38.768   
13  37.832  38.678  44.906  36.914  32.792  35.222  35.258  35.888  27.068   
14  33.944  31.442  34.754  32.432  30.542  34.610  28.634  29.336  23.828   
15  37.490  36.320  37.310  34.952  34.376  36.752  30.776  29.498  25.070   
16  40.928  42.998  42.476  38.390  31.640  37.382  36.500  31.028  30.236   
17  41.090  46.454  46.850  36.194  31.604  42.350  35.564  29.480  38.408   
18  40.442  45.680  48.578  33.782  32.306  48.704  36.464  34.592  40.658   
19  40.478  40.100  43.682  35.996  34.322  51.476  37.904  36.806  37.670   
20  41.342  40.712  48.452  38.732  38.696  51.548  40.730  35.132  35.114   
21  40.388  36.662  47.624  38.840  42.026  50.072  42.278  37.382  33.260   
22  46.058  39.272  39.776  40.172  41.324  50.972  41.396  40.406  34.016   
23  46.436  41.252  42.620  39.290  35.420  52.844  47.606  42.026  35.456   
24  43.844  43.412  38.876  41.324  40.568  53.816  49.334  40.532  39.092   
25  46.454  43.088  38.804  40.028  45.212  46.256  50.936  33.800  42.206   
26  45.212  42.188  39.326  38.210  48.812  44.294  46.292  32.558  45.014   
27  51.188  38.984  41.882  39.506  49.262  48.020  41.486  30.740  45.446   
28  55.616  35.114  40.334  39.398  49.532  50.576  42.242  37.040  43.934   

      1990  ...    2009    2010    2011    2012    2013    2014    2015  \
0   47.786  ...  46.562  43.124  46.922  53.348  35.204  45.572  39.362   
1   48.380  ...  48.380  48.470  43.322  50.342  33.314  43.448  44.492   
2   48.380  ...  49.226  51.674  45.302  47.480  34.340  45.518  36.518   
3   47.732  ...  50.810  47.408  46.706  45.572  36.284  47.030  34.214   
4   44.438  ...  43.286  45.482  37.706  44.978  38.984  49.568  35.006   
5   38.210  ...  43.394  49.010  39.920  46.274  37.958  52.052  37.598   
6   35.366  ...  45.896  43.970  41.036  46.634  38.390  51.962  41.162   
7   37.868  ...  45.122  42.224  34.736  43.934  40.676  49.010  43.502   
8   34.520  ...  42.908  44.132  36.068  42.512  37.148  41.810  38.624   
9   35.222  ...  40.640  43.376  35.762  37.436  43.664  37.310  43.448   
10  38.156  ...  40.244  42.062  36.986  37.652  45.464  36.518  48.398   
11  39.056  ...  41.396  43.448  37.112  41.432  40.334  37.796  43.106   
12  28.886  ...  35.294  43.646  40.586  46.976  31.208  38.606  39.974   
13  21.848  ...  39.002  32.792  36.374  43.538  29.714  36.194  35.618   
14  17.186  ...  30.020  32.216  31.190  33.170  28.562  32.900  33.242   
15  21.416  ...  30.434  34.970  35.852  36.302  28.598  34.304  33.440   
16  26.996  ...  35.510  38.912  38.372  41.072  31.838  35.006  34.394   
17  31.694  ...  41.396  38.102  44.942  43.592  38.138  36.950  33.728   
18  34.304  ...  46.616  36.320  47.282  45.212  42.692  40.172  35.186   
19  35.204  ...  47.246  37.328  43.556  44.762  43.214  45.590  34.862   
20  35.852  ...  41.324  40.892  47.588  40.262  43.952  50.216  33.512   
21  34.106  ...

In [8]:
# Define a function to annual minimum and maximum temperature time series:
def plot_daily_min_max_time_series(ba_to_plot: str, image_output_dir: str, image_resolution: int, save_images=False):
    
    # Compute the coldest and hottest days matrix using the function created above:
    coldest_output_df, hottest_output_df = process_daily_min_max_time_series(data_input_dir = data_input_dir,
                                                                             data_output_dir = data_output_dir, 
                                                                             ba_to_process = ba_to_plot)
    
    t_min_mean = coldest_output_df.dropna().values.mean().round(2)
    t_max_mean = hottest_output_df.dropna().values.mean().round(2)
     
    # Make the plot:
    plt.figure(figsize=(20, 14))
    plt.rcParams['font.size'] = 12
    
    plt.subplot(211)
    for year in range(1981,2019,1):
        if year != 2018:
           plt.plot(np.arange(-14.5, 14.5, 1, dtype=int), hottest_output_df[str(year)], color='r', linestyle='-', linewidth=0.35)
        else:
           plt.plot(np.arange(-14.5, 14.5, 1, dtype=int), hottest_output_df[str(year)], color='r', linestyle='-', linewidth=5)
    plt.plot([-14, 14], [t_max_mean, t_max_mean], color='gray', linestyle='--', linewidth=4)
    plt.plot(np.arange(-14.5, 14.5, 1, dtype=int), hottest_output_df.mean(axis=1), color='k', linestyle='-', linewidth=5)
    plt.xlim([-14, 14])
    plt.xticks(np.arange(-14.5, 14.5, 1, dtype=int),['-14','-13','-12','-11','-10','-9','-8','-7','-6','-5','-4','-3','-2','-1','$t_{0}$','+1','+2','+3','+4','+5','+6','+7','+8','+9','+10','+11','+12','+13','+14'])
    plt.ylim([hottest_output_df.dropna().values.min()-2, hottest_output_df.dropna().values.max()+2])
    plt.grid(True)
    plt.xlabel('Days from the Hottest Temperature of the Year');
    plt.ylabel('Daily Maximum Temperature [$^\circ$F]')
    plt.title(('Heat Wave Evolution from 1980-2019: ' + ba_to_plot))
    
    plt.subplot(212)
    for year in range(1981,2019,1):
        plt.plot(np.arange(-14.5, 14.5, 1, dtype=int), coldest_output_df[str(year)], color='b', linestyle='-', linewidth=0.35)
    plt.plot([-14, 14], [t_min_mean, t_min_mean], color='gray', linestyle='--', linewidth=4)
    plt.plot(np.arange(-14.5, 14.5, 1, dtype=int), coldest_output_df.mean(axis=1), color='k', linestyle='-', linewidth=5)
    plt.xlim([-14, 14])
    plt.xticks(np.arange(-14.5, 14.5, 1, dtype=int),['-14','-13','-12','-11','-10','-9','-8','-7','-6','-5','-4','-3','-2','-1','$t_{0}$','+1','+2','+3','+4','+5','+6','+7','+8','+9','+10','+11','+12','+13','+14'])
    plt.ylim([coldest_output_df.dropna().values.min()-2, coldest_output_df.dropna().values.max()+2])
    plt.grid(True)
    plt.xlabel('Days from the Coldest Temperature of the Year');
    plt.ylabel('Daily Minimum Temperature [$^\circ$F]')
    plt.title(('Cold Snap Evolution from 1980-2019: ' + ba_to_plot))
    
    # If the "save_images" flag is set to true then save the plot to a .png file:
    if save_images == True:
       filename = (ba_to_plot + '_Heat_Wave_Cold_Snap_Duration.png')
       plt.savefig(os.path.join(image_output_dir, filename), dpi=image_resolution, bbox_inches='tight', facecolor='white')


In [11]:
plot_daily_min_max_time_series(ba_to_plot = 'LDWP',
                               image_output_dir = image_output_dir, 
                               image_resolution = 300, 
                               save_images = True)
